# Week 11 Exercise

In this notebook exercise, we will have three different cases to practice your learning outcomes in the synchronous session. The demono notebook is available in this link. Please don't worry if you can finish this in class, but please remember to submit your final notebook in the following submission point. It is important for your to receive feedback about this notebook from the instructor as this is helpful in supporting your assignment. 

In [ ]:
# Importing all libries we used in this notebook

import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor # using package of testing VIF in statsmodels
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import numpy as np

## Case 1 - MBA Data

<center><img src="../Image/businessweek.png" width=600 height=400 /></center>

Consider the data for Business Week’s top U.S. MBA programs in `Excercise_1.csv`. The csv file is available in the data folder of the repo, so please use similar document path to _read_ the csv file. 

Use these data to estimate a multiple regression model to assess whether there is a relationship between the enroll-ment and the following explanatory variables: (a) thepercentage of international students, (b) the percentageof female students, (c) the percentage of Asian Americanstudents, (d) the percentage of minority students, and (e) the resident tuition and fees at these business schools.

Hints of the data analysis process:

1. Loading the data from correct document/folder path - `ExData`
2. Data preparation - renaming variable and cleaning the NaN
3. Data analysis - using the sm regression method not the smf with formula api
4. Data interpretation - check the demo notebook to interpret the VIF values

In [ ]:
# 1 loading the data to a new dataframe you may named it as df_MBA (Solution)
df_MBA = pd.read_csv('../ExData/Exercise_1.csv')
df_MBA.head()

In [ ]:
# 2 renaming the variable names that are required in this case study (solution)
df_MBA = df_MBA.rename(columns={'Pct International':'Pct_International', 
                                'Pct Female':'Pct_Female', 
                                'Pct Asian American':'Pct_Asian_American',
                                'Pct Minority':'Pct_Minority', 'Resident Tuition, Fees':'Resident_Tuition'})
df_MBA.head()

In [ ]:
# 3 droping the missing values use dropna function to the dataframe (solution)
df_MBA = df_MBA.dropna()

In [ ]:
# 4a building your regression model via the new approch I introduced (solution)

y = df_MBA.Enrollment
X = df_MBA[["Pct_International","Pct_Female","Pct_Asian_American","Pct_Minority","Resident_Tuition"]].assign(const=1)

results = sm.OLS(y, X).fit()
print(results.summary())

In [ ]:
# 4b calculating the VIF values

# using the vif function to obtain the vif value of the independent variables

vif = [variance_inflation_factor(exog=X.values, exog_idx=i) for i in range(X.shape[1])]

# creating a new dataframe to report the results
vif_table = pd.DataFrame({'coef_name': X.columns, 'vif': np.around(vif,3)})
print(vif_table)

### Question 1
Determine whether each of the regression coefficientsfor the explanatory variables in this model is statisti-cally different from zero at the5%significance level.Summarize your findings.

Please type in you answer below:

_______________________________________________

### Question 2
Is there evidence of multicollinearity in this model? Explain why or why not?

Please type in you answer below:

_______________________________________________

## Case 2 Photography Shop (Time series data)

The file `Exercise_2.csv` contains monthly sales for a photography studio and the price charged per portrait duringeach month. 

Use regression to estimate an equation for predicting the current month’s sales from last month’s sales and the current month’s price. 

In this example, we will experience a new method to make the prediction better for the 'time-lagged' effect. Considering the influence of certain practices or company's investment will take time, the effect of independent variables could be lagged. To estimate such effect in the regression model, we can create a lagged variable for the analysis. To create a lagged variable in pandas, we can use the `shift` function. 

For example, if we have a dataframe called `df` and we want to lag the variable of `Satisfaction` for 1 index (all the observations on Satisfaction will lag one row), we can use the below syntax:

`df['lagSatisfaction']=df['Satisfaction'].shift(1)`

In [ ]:
df_Photo = pd.read_csv('../ExData/Exercise_2.csv')

# Using the above example to create a time lagged variable (solution)
# Hints - check observation format and drop NaN

def format(Format):
    return(int(Format.replace('$','').replace(',','')))

df_Photo['Sales'] = df_Photo['Sales'].apply(format)
df_Photo['LagSales'] = df_Photo['Sales'].shift(1) 
df_Photo['Price'] = df_Photo['Price'].apply(format)

df_Photo=df_Photo.dropna()

df_Photo

In [ ]:
# Building a regression model for predicting the current month's sales from last month's sales and current month's price

y = df_Photo.Sales
X = df_Photo[["Price","LagSales"]].assign(const=1)

results = sm.OLS(y, X).fit()
print(results.summary())

### Question 1

Discuss how you can tell whether multicollinearity, or heteroscedasticity might be aproblem. Please refer to the Week 11 Lecture Demo - 1.2.1 Assumption 3 and 2.0 Multicollinearity.

Please type in you answer below:

_______________________________________________

In [ ]:
# Using residual plot to check the heteroscedasticity

sns.histplot(results.resid)

In [ ]:
# Checking the VIF again

# using the vif function to obtain the vif value of the independent variables

vif = [variance_inflation_factor(exog=X.values, exog_idx=i) for i in range(X.shape[1])]

# creating a new dataframe to report the results

vif_table = pd.DataFrame({'coef_name': X.columns, 'vif': np.around(vif,3)})
print(vif_table)

### Question 2

If the price of a portrait during month 21 is $30, predict month 21 sales.

Please type in you answer below:

_______________________________________________

In [ ]:
# (Solution)
# Hint: creating a new dataframe with Price value and LagSales value and assigning a constant values

data = {'Price':30,
        'LagSales':907000},
df = pd.DataFrame(data).assign(const=1)

results.predict(df)